# Deploy machine learning models to Azure

description: (preview) deploy your machine learning or deep learning model as a web service in the Azure cloud.

# Connect to your workspace

In [1]:
from azureml.core import Workspace

ws = Workspace.from_config()
ws

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


Workspace.create(name='chrjia-eastus2euap2', subscription_id='13e50845-67bc-4ac5-94db-48d493a6d9e8', resource_group='chrjia-rg')

# Register your model

A registered model is a logical container stored in the cloud, containing all files located at `model_path`, which is associated with a version number and other metadata.



## Register a model from a local file

You can register a model by providing the local path of the model. You can provide the path of either a folder or a single file on your local machine.

In [2]:
import urllib.request
from azureml.core.model import Model
# Download model
urllib.request.urlretrieve("https://aka.ms/bidaf-9-model", 'model.onnx')

# Register model
model = Model.register(ws, model_name='bidaf_onnx', model_path='./model.onnx')

Registering model bidaf_onnx


# Define an inference configuration

The inference configuration below specifies that the machine learning deployment will use the file echo_score.py in the ./source_dir directory to process incoming requests and that it will use the Docker image with the Python packages specified in the project_environment environment.

In [3]:
from azureml.core import Environment
from azureml.core.model import InferenceConfig

env = Environment(name='project_environment')
inference_config = InferenceConfig(environment=env, source_directory='./source_dir', entry_script='./echo_score.py')

## Define a deployment configuration

In [4]:
from azureml.core.webservice import LocalWebservice

deployment_config = LocalWebservice.deploy_configuration(port=6789)

## Deploy your machine learning model

A deployment configuration specifies the amount of memory and cores to reserve for your webservice will require in order to run, as well as configuration details of the underlying webservice. For example, a deployment configuration lets you specify that your service needs 2 gigabytes of memory, 2 CPU cores, 1 GPU core, and that you want to enable autoscaling.

The options available for a deployment configuration differ depending on the compute target you choose. In a local deployment, all you can specify is which port your webservice will be served on.

In [6]:
service = Model.deploy(ws, "myservice", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output=True)

ModelNotFound: Model with id bidaf_onnx:14 not found in provided workspace



Generating Docker build context.
Package creation Succeeded
Logging into Docker registry chrjiaeastusc6ba5291.azurecr.io
Logging into Docker registry chrjiaeastusc6ba5291.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM chrjiaeastusc6ba5291.azurecr.io/azureml/azureml_673d52b5abe224c83574bc767a17fa28
 ---> 0884414da052
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> debfdd287ac0
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6IjEzZTUwODQ1LTY3YmMtNGFjNS05NGRiLTQ4ZDQ5M2E2ZDllOCIsInJlc291cmNlR3JvdXBOYW1lIjoiY2hyamlhLXJnIiwiYWNjb3VudE5hbWUiOiJjaHJqaWEtZWFzdHVzMmV1YXAyIiwid29ya3NwYWNlSWQiOiI1OTkzZGVjMS01OTczLTQ4NTktOGJlMS03MDczMzJjNTQ5NWQifSwibW9kZWxzIjp7fSwibW9kZWxzSW5mbyI6e319 | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in e092a63d1b2d
 ---> d6883fbeadd8
Step 4/5 : RUN mv '/var/azureml-app/tmp67_pkw3d.py' /var/azureml-app/main.py
 ---> Running in 6dee06319cbb
 ---> 10994c7c94b9
Step 5/5 : 

In [7]:
print(service.get_logs())

2021-04-30T12:53:00,348883000+00:00 - gunicorn/run 
2021-04-30T12:53:00,348822500+00:00 - rsyslog/run 
2021-04-30T12:53:00,348882800+00:00 - iot-server/run 
2021-04-30T12:53:00,348847400+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_da3e97fcb51801118b8e80207f3e01ad/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_da3e97fcb51801118b8e80207f3e01ad/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_da3e97fcb51801118b8e80207f3e01ad/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_da3e97fcb51801118b8e80207f3e01ad/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_da3e97fcb51801118b8e80207f3e01ad/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
EdgeHubC

## Call into your model

In [8]:
import requests
import json

uri = service.scoring_uri
requests.get('http://localhost:6789')
headers = {'Content-Type': 'application/json'}
data = {"query": "What color is the fox", "context": "The quick brown fox jumped over the lazy dog."}
data = json.dumps(data)
response = requests.post(uri, data=data, headers=headers)
print(response.json())

test is {'query': 'What color is the fox', 'context': 'The quick brown fox jumped over the lazy dog.'}


Notice the use of the AZUREML_MODEL_DIR environment variable to locate your registered model. Now that you've added some pip packages, you also need to update your inference configuration to add in those additional packages

In [9]:
env = Environment(name='myenv')
python_packages = ['nltk', 'numpy', 'onnxruntime']
for package in python_packages:
    env.python.conda_dependencies.add_pip_package(package)

inf_config = InferenceConfig(environment=env, source_directory='./source_dir', entry_script='./score.py')

## Deploy again and call your service

In [10]:
service = Model.deploy(ws, "myservice", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output=True)

Generating Docker build context.
Package creation Succeeded
Logging into Docker registry chrjiaeastusc6ba5291.azurecr.io
Logging into Docker registry chrjiaeastusc6ba5291.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM chrjiaeastusc6ba5291.azurecr.io/azureml/azureml_673d52b5abe224c83574bc767a17fa28
 ---> 0884414da052
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> 6ac5a662619a
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6IjEzZTUwODQ1LTY3YmMtNGFjNS05NGRiLTQ4ZDQ5M2E2ZDllOCIsInJlc291cmNlR3JvdXBOYW1lIjoiY2hyamlhLXJnIiwiYWNjb3VudE5hbWUiOiJjaHJqaWEtZWFzdHVzMmV1YXAyIiwid29ya3NwYWNlSWQiOiI1OTkzZGVjMS01OTczLTQ4NTktOGJlMS03MDczMzJjNTQ5NWQifSwibW9kZWxzIjp7fSwibW9kZWxzSW5mbyI6e319 | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in d6d7f4ac3c32
 ---> 4f3ab2e502bc
Step 4/5 : RUN mv '/var/azureml-app/tmp9losnro_.py' /var/azureml-app/main.py
 ---> Running in 146dfe834545
 ---> 851e8cab14b3
Step 5/5 : 

In [11]:
print(service.get_logs())

2021-04-30T12:58:05,078164200+00:00 - iot-server/run 
2021-04-30T12:58:05,078539600+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_da3e97fcb51801118b8e80207f3e01ad/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_da3e97fcb51801118b8e80207f3e01ad/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_da3e97fcb51801118b8e80207f3e01ad/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_da3e97fcb51801118b8e80207f3e01ad/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_da3e97fcb51801118b8e80207f3e01ad/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2021-04-30T12:58:05,079400900+00:00 - gunicorn/run 
2021-04-30T12:58:05,084895700+00:00 - rsyslog/run 
EdgeHubC

Then ensure you can send a post request to the service:

In [12]:
import requests

uri = service.scoring_uri

headers = {'Content-Type': 'application/json'}
data = {"query": "What color is the fox", "context": "The quick brown fox jumped over the lazy dog."}
data = json.dumps(data)
response = requests.post(uri, data=data, headers=headers)
print(response.json())

test is {'query': 'What color is the fox', 'context': 'The quick brown fox jumped over the lazy dog.'}


## Re-deploy to cloud

Once you've confirmed your service works locally and chosen a remote compute target, you are ready to deploy to the cloud.

Change your deploy configuration to correspond to the compute target you've chosen, in this case Azure Container Instances.

In [13]:
from azureml.core.webservice import AciWebservice

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 0.5, memory_gb = 1, auth_enabled=True)

Deploy your service again

In [14]:
service = Model.deploy(ws, "myservice", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-04-30 18:30:38+05:30 Creating Container Registry if not exists.
2021-04-30 18:30:39+05:30 Registering the environment.
2021-04-30 18:30:40+05:30 Use the existing image.
2021-04-30 18:30:40+05:30 Generating deployment configuration.
2021-04-30 18:30:42+05:30 Submitting deployment to compute..
2021-04-30 18:31:30+05:30 Checking the status of deployment testservice1..
2021-04-30 18:32:53+05:30 Checking the status of inference endpoint testservice1.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [15]:
print(service.get_logs())

2021-04-30T13:02:51,581997500+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_da3e97fcb51801118b8e80207f3e01ad/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_da3e97fcb51801118b8e80207f3e01ad/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_da3e97fcb51801118b8e80207f3e01ad/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_da3e97fcb51801118b8e80207f3e01ad/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_da3e97fcb51801118b8e80207f3e01ad/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2021-04-30T13:02:51,584386300+00:00 - gunicorn/run 
2021-04-30T13:02:51,607040800+00:00 - iot-server/run 
2021-04-30T13:02:51,616880500+00:00 - rsyslog/run 
EdgeHubC

## Call your remote webservice

When you deploy remotely, you may have key authentication enabled. The example below shows how to get your service key with Python in order to make an inference request.

In [16]:
import requests
import json
from azureml.core import Webservice

service = Webservice(workspace=ws, name='myservice')
scoring_uri = service.scoring_uri

# If the service is authenticated, set the key or token
key, _ = service.get_keys()

# Set the appropriate headers
headers = {'Content-Type': 'application/json'}
headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response and logs
data = {"query": "What color is the fox", "context": "The quick brown fox jumped over the lazy dog."}
data = json.dumps(data)
resp = requests.post(scoring_uri, data=data, headers=headers)
print(resp.text)

"test is {'query': 'What color is the fox', 'context': 'The quick brown fox jumped over the lazy dog.'}"


In [17]:
print(service.get_logs())

2021-04-30T13:02:51,581997500+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_da3e97fcb51801118b8e80207f3e01ad/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_da3e97fcb51801118b8e80207f3e01ad/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_da3e97fcb51801118b8e80207f3e01ad/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_da3e97fcb51801118b8e80207f3e01ad/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_da3e97fcb51801118b8e80207f3e01ad/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2021-04-30T13:02:51,584386300+00:00 - gunicorn/run 
2021-04-30T13:02:51,607040800+00:00 - iot-server/run 
2021-04-30T13:02:51,616880500+00:00 - rsyslog/run 
EdgeHubC

## Delete resources

In [18]:
service.delete()
model.delete()

## Next Steps

Try reading [our documentation](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-and-where?tabs=python)